In [1]:
from collections import defaultdict, Counter
from functools import reduce
import glob

import pandas as pd
import simplejson as json

In [2]:
words_filename = '../words/words.csv'

In [3]:
def wordlist_to_wordset(word_list):
    words = []
    for word_group in word_list:
        if type(word_group) is str:
            words.extend([c.strip() for c in word_group.split('/')])
    return set(words)

def build_tw_cn_dict(tw_word_list, cn_word_list):
    
    tw2cn = defaultdict(list)
    cn2tw = defaultdict(list)
    
    for tw_word_group, cn_word_group in zip(tw_word_list, cn_word_list):
        if type(tw_word_group) is str:
            tw_words = [c.strip() for c in tw_word_group.split('/')]
        else:
            tw_words = [None]
        
        if type(cn_word_group) is str:
            cn_words = [c.strip() for c in cn_word_group.split('/')]
        else:
            tw_words = [None]
            
        for tw_word in tw_words:
            for cn_word in cn_words:
                tw2cn[tw_word].append(cn_word)
                cn2tw[cn_word].append(tw_word)
    
    return tw2cn, cn2tw

def new_defined_wordset(tw_word_list, cn_word_list):
    
    wordset = set()
    for tw_word_group, cn_word_group in zip(tw_word_list, cn_word_list):
        if type(tw_word_group) is str and type(cn_word_group) is str:
            tw_words = [c.strip() for c in tw_word_group.split('/')]
            cn_words = set([c.strip() for c in cn_word_group.split('/')])
            for cn_word in cn_words:
                for tw_word in tw_words:
                    if cn_word in tw_word:
                        break
                else:
                    wordset.add(cn_word)
    
    wordset -= wordlist_to_wordset(tw_word_list)
    return wordset

df = pd.read_csv(words_filename)
cn_word_list = df['cn_word'].tolist()
tw_word_list = df['tw_word'].tolist()
cn_word_set = wordlist_to_wordset(cn_word_list)
tw_word_set = wordlist_to_wordset(tw_word_list)
# filtered_word_set = cn_word_set - tw_word_set
filtered_word_set = new_defined_wordset(tw_word_list, cn_word_list)
tw2cn, cn2tw = build_tw_cn_dict(tw_word_list, cn_word_list)

In [4]:
def find_cn_words(article, word_set):
    cn_word_count = Counter()
    for cn_word in word_set:
        if cn_word in article:
            cn_word_count[cn_word] += 1
    return cn_word_count

In [5]:
freq_counter = Counter()
for filename in glob.glob('../news/*/*/*'):
    with open(filename, 'r') as f:
        article = json.load(f)
        result = find_cn_words(article['content'], filtered_word_set)
        if result:
            print(article['url'], result)
            freq_counter += result

https://www.ettoday.net/news/20190731/1502216.htm Counter({'富二代': 1})
https://www.ettoday.net/news/20190731/1502596.htm Counter({'保安': 1})
https://www.ettoday.net/news/20190731/1480451.htm Counter({'耳機': 1})
https://www.ettoday.net/news/20190731/1503050.htm Counter({'定點': 1})
https://www.ettoday.net/news/20190731/1502248.htm Counter({'激光武器': 1, '激光': 1, '攝像': 1})
https://www.ettoday.net/news/20190731/1502969.htm Counter({'質檢': 1})
https://www.ettoday.net/news/20190731/1502528.htm Counter({'打的': 1})
https://www.ettoday.net/news/20190731/1502310.htm Counter({'主頁': 1})
https://www.ettoday.net/news/20190731/1502578.htm Counter({'公安局': 1, '個人遊': 1})
https://www.ettoday.net/news/20190731/1502535.htm Counter({'網絡': 1})
https://www.ettoday.net/news/20190731/1502320.htm Counter({'坎': 1})
https://www.ettoday.net/news/20190731/1502375.htm Counter({'摩': 1, '運營': 1})
https://www.ettoday.net/news/20190731/1503033.htm Counter({'本體': 1})
https://www.ettoday.net/news/20190731/1502522.htm Counter({'電商':

https://www.ettoday.net/news/20190730/1501944.htm Counter({'電大': 1})
https://www.ettoday.net/news/20190730/1502024.htm Counter({'打女': 1})
https://www.ettoday.net/news/20190730/1501794.htm Counter({'微機': 1, '接手': 1})
https://www.ettoday.net/news/20190730/1501343.htm Counter({'的士': 1})
https://www.ettoday.net/news/20190730/1501312.htm Counter({'診室': 1})
https://www.ettoday.net/news/20190730/1501575.htm Counter({'摩': 1})
https://www.ettoday.net/news/20190730/1501662.htm Counter({'摩': 1})
https://www.ettoday.net/news/20190730/1501325.htm Counter({'奔馳': 1})
https://www.ettoday.net/news/20190730/1501762.htm Counter({'基站': 1, '運營': 1})
https://www.ettoday.net/news/20190730/1501455.htm Counter({'運力': 1})
https://www.ettoday.net/news/20190730/1502026.htm Counter({'互聯網': 1, '電商': 1})
https://www.ettoday.net/news/20190730/1501498.htm Counter({'行車': 1, '基站': 1, '電動車': 1})
https://www.ettoday.net/news/20190730/1501434.htm Counter({'摩': 1})
https://www.ettoday.net/news/20190730/1502124.htm Counter({

https://www.setn.com/News.aspx?NewsID=575218 Counter({'下網': 1})
https://www.setn.com/News.aspx?NewsID=575073 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=575208 Counter({'定標': 1})
https://www.setn.com/News.aspx?NewsID=575400 Counter({'保安': 1})
https://www.setn.com/News.aspx?NewsID=575396 Counter({'打的': 1})
https://www.setn.com/News.aspx?NewsID=575225 Counter({'認領': 1})
https://www.setn.com/News.aspx?NewsID=575307 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=575352 Counter({'保安': 1})
https://www.setn.com/News.aspx?NewsID=574932 Counter({'接手': 1})
https://www.setn.com/News.aspx?NewsID=575408 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=575061 Counter({'電動車': 1})
https://www.setn.com/News.aspx?NewsID=573410 Counter({'信息': 1, '軟件': 1, '攝像': 1})
https://www.setn.com/News.aspx?NewsID=573533 Counter({'信息': 1})
https://www.setn.com/News.aspx?NewsID=575308 Counter({'咖喱': 1})
https://www.setn.com/News.aspx?NewsID=575151 Counter({'內帶': 1})
https://www.setn.com/New

https://www.setn.com/News.aspx?NewsID=592922 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=592892 Counter({'人流': 1})
https://www.setn.com/News.aspx?NewsID=592661 Counter({'攝像': 1})
https://www.setn.com/News.aspx?NewsID=593096 Counter({'罰款': 1, '摩': 1})
https://www.setn.com/News.aspx?NewsID=592667 Counter({'人流': 1})
https://www.setn.com/News.aspx?NewsID=592670 Counter({'的士': 1})
https://www.setn.com/News.aspx?NewsID=592691 Counter({'青茶': 1})
https://www.setn.com/News.aspx?NewsID=592986 Counter({'套服': 1})
https://www.setn.com/News.aspx?NewsID=593148 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=593153 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=592904 Counter({'班車': 1})
https://www.setn.com/News.aspx?NewsID=592959 Counter({'定點': 1})
https://www.setn.com/News.aspx?NewsID=592909 Counter({'人流': 1})
https://www.setn.com/News.aspx?NewsID=592379 Counter({'蒙藥': 1})
https://www.setn.com/News.aspx?NewsID=589988 Counter({'信息': 1, '電商': 1})
https://www.setn.com/News.

https://www.setn.com/News.aspx?NewsID=638978 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=639354 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=639259 Counter({'外帶': 1, '行車': 1})
https://www.setn.com/News.aspx?NewsID=638891 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=638991 Counter({'診室': 1})
https://www.setn.com/News.aspx?NewsID=638689 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=639157 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=638511 Counter({'輸液': 1})
https://www.setn.com/News.aspx?NewsID=639168 Counter({'定點': 1})
https://www.setn.com/News.aspx?NewsID=639448 Counter({'富二代': 1})
https://www.setn.com/News.aspx?NewsID=639374 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=639014 Counter({'打的': 1})
https://www.setn.com/News.aspx?NewsID=639316 Counter({'說唱': 1, '摩': 1})
https://www.setn.com/News.aspx?NewsID=639078 Counter({'電商': 1})
https://www.setn.com/News.aspx?NewsID=639214 Counter({'高清': 1})
https://www.setn.com/News.as

...

https://www.setn.com/News.aspx?NewsID=619752 Counter({'水地': 1})
https://www.setn.com/News.aspx?NewsID=619756 Counter({'網絡': 1})
https://www.setn.com/News.aspx?NewsID=619869 Counter({'電眼': 1})
https://www.setn.com/News.aspx?NewsID=619957 Counter({'信息': 1})
https://www.setn.com/News.aspx?NewsID=619590 Counter({'極差': 1})
https://www.setn.com/News.aspx?NewsID=571376 Counter({'打的': 1})
https://www.setn.com/News.aspx?NewsID=571562 Counter({'說唱': 1})
https://www.setn.com/News.aspx?NewsID=570040 Counter({'信息': 1})
https://www.setn.com/News.aspx?NewsID=571687 Counter({'保安': 1})
https://www.setn.com/News.aspx?NewsID=571696 Counter({'沙包': 1, '抽水': 1})
https://www.setn.com/News.aspx?NewsID=571603 Counter({'抽水': 1})
https://www.setn.com/News.aspx?NewsID=571430 Counter({'保安': 1})
https://www.setn.com/News.aspx?NewsID=570391 Counter({'暖風機': 1})
https://www.setn.com/News.aspx?NewsID=571673 Counter({'診室': 1})
https://www.setn.com/News.aspx?NewsID=570486 Counter({'電商': 1})
https://www.setn.com/News.aspx

https://www.setn.com/News.aspx?NewsID=582029 Counter({'坎': 1})
https://www.setn.com/News.aspx?NewsID=581688 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=581981 Counter({'打的': 1})
https://www.setn.com/News.aspx?NewsID=582036 Counter({'水地': 1})
https://www.setn.com/News.aspx?NewsID=581996 Counter({'耳機': 1})
https://www.setn.com/News.aspx?NewsID=581692 Counter({'接手': 1})
https://www.setn.com/News.aspx?NewsID=581708 Counter({'耳機': 1})
https://www.setn.com/News.aspx?NewsID=581772 Counter({'打的': 1})
https://www.setn.com/News.aspx?NewsID=582045 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=581659 Counter({'網絡': 1})
https://www.setn.com/News.aspx?NewsID=581744 Counter({'公安局': 1, '網民': 1})
https://www.setn.com/News.aspx?NewsID=582019 Counter({'電商': 1})
https://www.setn.com/News.aspx?NewsID=581453 Counter({'打的': 1})
https://www.setn.com/News.aspx?NewsID=581976 Counter({'匝道': 1, '電動車': 1})
https://www.setn.com/News.aspx?NewsID=581795 Counter({'打的': 1, '坎': 1})
https://www.set

https://www.setn.com/News.aspx?NewsID=638564 Counter({'大專': 1})
https://www.setn.com/News.aspx?NewsID=606419 Counter({'人流': 1})
https://www.setn.com/News.aspx?NewsID=606400 Counter({'網絡': 1, '筆記本': 1})
https://www.setn.com/News.aspx?NewsID=606299 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=606668 Counter({'比索': 1})
https://www.setn.com/News.aspx?NewsID=606570 Counter({'筆記本': 1})
https://www.setn.com/News.aspx?NewsID=606381 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=606671 Counter({'漢堡包': 1})
https://www.setn.com/News.aspx?NewsID=606344 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=606443 Counter({'奔馳': 1})
https://www.setn.com/News.aspx?NewsID=606336 Counter({'接手': 1})
https://www.setn.com/News.aspx?NewsID=606611 Counter({'行車': 1})
https://www.setn.com/News.aspx?NewsID=604046 Counter({'航天': 1})
https://www.setn.com/News.aspx?NewsID=606324 Counter({'比特': 1})
https://www.setn.com/News.aspx?NewsID=606314 Counter({'摩': 1})
https://www.setn.com/News.aspx?N

https://www.setn.com/News.aspx?NewsID=594261 Counter({'行車': 1})
https://www.setn.com/News.aspx?NewsID=593844 Counter({'社交網站': 1})
https://www.setn.com/News.aspx?NewsID=594392 Counter({'本體': 1})
https://www.setn.com/News.aspx?NewsID=594369 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=593820 Counter({'富二代': 1})
https://www.setn.com/News.aspx?NewsID=594184 Counter({'套服': 1, '摩': 1})
https://www.setn.com/News.aspx?NewsID=593959 Counter({'的士': 1})
https://www.setn.com/News.aspx?NewsID=594347 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=594204 Counter({'旅遊車': 1})
https://www.setn.com/News.aspx?NewsID=594189 Counter({'大考': 1})
https://www.setn.com/News.aspx?NewsID=594197 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=594146 Counter({'亞行': 1})
https://www.setn.com/News.aspx?NewsID=593936 Counter({'站點': 1})
https://www.setn.com/News.aspx?NewsID=594212 Counter({'人流': 1})
https://www.setn.com/News.aspx?NewsID=594022 Counter({'行車': 1})
https://www.setn.com/News.aspx?

https://www.setn.com/News.aspx?NewsID=640625 Counter({'高清': 1})
https://www.setn.com/News.aspx?NewsID=640680 Counter({'套服': 1})
https://www.setn.com/News.aspx?NewsID=640366 Counter({'行車': 1})
https://www.setn.com/News.aspx?NewsID=640738 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=640368 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=640852 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=640350 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=640451 Counter({'電商': 1})
https://www.setn.com/News.aspx?NewsID=639968 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=640365 Counter({'罰款': 1})
https://www.setn.com/News.aspx?NewsID=640425 Counter({'人流': 1})
https://www.setn.com/News.aspx?NewsID=640389 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=640892 Counter({'摩': 1})
https://www.setn.com/News.aspx?NewsID=640341 Counter({'電商': 1})
https://www.setn.com/News.aspx?NewsID=640750 Counter({'打的': 1})
https://www.setn.com/News.aspx?NewsID=640374 Co